In [1]:
import tensorflow as tf
from tensorflow.feature_column import embedding_column, numeric_column, categorical_column_with_identity
import numpy as np
import os

In [2]:
tf.enable_eager_execution()

In [2]:
def build_feature_columns():
    pickup_longitude = tf.feature_column.numeric_column('pickup_longitude')
    pickup_latitude = tf.feature_column.numeric_column('pickup_latitude')
    dropoff_longitude = tf.feature_column.numeric_column('dropoff_longitude')
    dropoff_latitude = tf.feature_column.numeric_column('dropoff_latitude')
    
    base_columns = [

    ]
    
    weekday = tf.feature_column.categorical_column_with_identity('weekday', 7)
    month = tf.feature_column.categorical_column_with_identity('month', 12)
    hour = tf.feature_column.categorical_column_with_identity('hour', 24)
    week_number = tf.feature_column.categorical_column_with_identity('week_number', 53)
    
    pickup_x = tf.feature_column.bucketized_column(pickup_longitude, list(np.linspace(0.0,1.0, 126)))
    pickup_y = tf.feature_column.bucketized_column(pickup_latitude, list(np.linspace(0.0,1.0, 124)))
    dropoff_x = tf.feature_column.bucketized_column(pickup_latitude, list(np.linspace(0.0,1.0, 126)))
    dropoff_y = tf.feature_column.bucketized_column(pickup_latitude, list(np.linspace(0.0,1.0, 124)))
    
    cross_columns = [
        tf.feature_column.crossed_column([pickup_x, pickup_y], 126*124),
        tf.feature_column.crossed_column([dropoff_x, dropoff_y], 126*124),
        tf.feature_column.crossed_column([month, weekday, hour], 7*12*24)
    ]
    
    wide_columns = base_columns + cross_columns
    
    distance = tf.feature_column.numeric_column('distance')
    weekday_embed = embedding_column(weekday, dimension=8)
    hour_embed = embedding_column(hour, dimension=8)
    week_number_embed = embedding_column(week_number, dimension=16)
    
    deep_columns = [
        distance,
        weekday_embed,
        hour_embed,
        week_number_embed
    ]
    
    return wide_columns, deep_columns

In [3]:
wide_columns, deep_columns = build_feature_columns()

In [4]:
def build_estimator(fc_fn):
    wide_columns, deep_columns = fc_fn()
    hidden_units = [100, 75, 50, 25]
    return tf.estimator.DNNLinearCombinedRegressor(
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units)
model = build_estimator(build_feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmpwic07upe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb262001d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
def parse_function(example_proto):
    column_description = {
         'fare_amount': tf.FixedLenFeature([], tf.float32),
            'pickup_longitude': tf.FixedLenFeature([], tf.float32),
            'pickup_latitude': tf.FixedLenFeature([], tf.float32),
            'dropoff_longitude': tf.FixedLenFeature([], tf.float32),
            'dropoff_latitude': tf.FixedLenFeature([], tf.float32),
            'distance': tf.FixedLenFeature([], tf.float32),
            'hour': tf.FixedLenFeature([], tf.int64),
            'month': tf.FixedLenFeature([], tf.int64),
            'week_number': tf.FixedLenFeature([], tf.int64),
            'weekday': tf.FixedLenFeature([], tf.int64)
    } 
    columns = tf.parse_single_example(example_proto, column_description)
    
    label = columns.pop('fare_amount')
    features = columns
    
    
    return features, label

In [18]:
dataset_dir = os.path.join('dataset')

train_data_file_name = os.path.join('train', 'train_dataset')
train_dataset_full_path = os.path.join(dataset_dir, train_data_file_name)

eval_data_file_name = os.path.join('eval', 'eval_dataset')
eval_dataset_full_path = os.path.join(dataset_dir, eval_data_file_name)

In [19]:
train_raw_dataset = tf.data.TFRecordDataset(train_dataset_full_path)
eval_raw_dataset = tf.data.TFRecordDataset(eval_dataset_full_path)

In [20]:
train_parsed_dataset = train_raw_dataset.map(parse_function)
eval_parsed_dataset = eval_raw_dataset.map(parse_function)

In [24]:
def input_fn(parsed_dataset):
    dataset = parsed_dataset.shuffle(1000)
    dataset = dataset.batch(512)
    dataset = dataset.repeat(100)
    
    iterator = dataset.make_one_shot_iterator()
    features, label = iterator.get_next()
    return (features, label)


In [25]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda : input_fn(train_parsed_dataset))
eval_spec = tf.estimator.EvalSpec(input_fn=lambda : input_fn(eval_parsed_dataset))

In [26]:
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/wt/dv0qlxcn5j9b0h9vd8tr23bh0000gn/T/tmpwic07upe/model.ckpt.


InvalidArgumentError: assertion failed: [4 7 2...] [12]
	 [[node linear/linear_model/linear_model/linear_model/hour_X_month_X_weekday/assert_less_than_num_buckets/Assert/AssertGuard/Assert (defined at /Users/bryanwu/anaconda/envs/Tensorflow/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/canned/linear.py:345) ]]

Caused by op 'linear/linear_model/linear_model/linear_model/hour_X_month_X_weekday/assert_less_than_num_buckets/Assert/AssertGuard/Assert', defined at:
  File "/Users/bryanwu/anaconda/envs/Tensorflow/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/bryanwu/anaconda/envs/Tensorflow/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/bryanwu/anaconda/envs/Tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/bryanwu/anaconda/envs/Tensorflow/lib/python3.7/site-packages/traitlets/config/application.py",